In [1]:
!pip install langchain pypdf markdown numpy
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 685.7 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 1.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.33
    Uninstalling langchain-core-0.3.33:
      Successfully uninstalled langchain-core-0.3.33
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.5
    Uninstalling langchain-text-splitters-0.3.5:
      Successfully uninstalled langchain-text-splitters-0.3.5
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.17
    Uninstalling langchain-0.3.17:
      Successfully uninstalled langchain-0.3.17
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.3.16
    Uninstalling langchain-community-0.3.16:
      Successfully uninstalled langchain-community-0.3.16


In [2]:
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.schema import Document
import glob

#### Reading Dataset

pdf_files = glob.glob("data/*.pdf")
md_files = glob.glob("data/*.md")
raw_data=[]
for file in pdf_files:     # Load a PDF files
  pdf_loader = PyPDFLoader(file)
  pdf_documents = pdf_loader.load()
  raw_data.extend(pdf_documents)
for file in md_files:# Load a Markdown file
  md_loader = TextLoader(file, encoding="utf-8")
  md_documents = md_loader.load()
  raw_data.extend(md_documents)

In [3]:
for data in raw_data[0:3]:
  print(data.page_content)

DeepSeek-R1: Incentivizing Reasoning Capability in LLMs via
Reinforcement Learning
DeepSeek-AI
research@deepseek.com
Abstract
We introduce our first-generation reasoning models, DeepSeek-R1-Zero and DeepSeek-R1.
DeepSeek-R1-Zero, a model trained via large-scale reinforcement learning (RL) without super-
vised fine-tuning (SFT) as a preliminary step, demonstrates remarkable reasoning capabilities.
Through RL, DeepSeek-R1-Zero naturally emerges with numerous powerful and intriguing
reasoning behaviors. However, it encounters challenges such as poor readability, and language
mixing. To address these issues and further enhance reasoning performance, we introduce
DeepSeek-R1, which incorporates multi-stage training and cold-start data before RL. DeepSeek-
R1 achieves performance comparable to OpenAI-o1-1217 on reasoning tasks. To support the
research community, we open-source DeepSeek-R1-Zero, DeepSeek-R1, and six dense models
(1.5B, 7B, 8B, 14B, 32B, 70B) distilled from DeepSeek-R1 based o

In [4]:
########## Generate Chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize a text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,  # Maximum size of each chunk
    chunk_overlap=60  # Overlap between chunks to maintain context
)

# Split the documents into chunks
document_chunks = text_splitter.split_documents(raw_data)


for chunk in document_chunks[:3]:  # Print the first 5 chunks
    print("Chunk Content:", chunk.page_content)
    print("Metadata:", chunk.metadata)



Chunk Content: DeepSeek-R1: Incentivizing Reasoning Capability in LLMs via
Reinforcement Learning
DeepSeek-AI
research@deepseek.com
Abstract
We introduce our first-generation reasoning models, DeepSeek-R1-Zero and DeepSeek-R1.
DeepSeek-R1-Zero, a model trained via large-scale reinforcement learning (RL) without super-
vised fine-tuning (SFT) as a preliminary step, demonstrates remarkable reasoning capabilities.
Metadata: {'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-01-23T01:45:31+00:00', 'author': '', 'keywords': '', 'moddate': '2025-01-23T01:45:31+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'data/2501.12948v1.pdf', 'total_pages': 22, 'page': 0, 'page_label': '1'}
Chunk Content: Through RL, DeepSeek-R1-Zero naturally emerges with numerous powerful and intriguing
reasoning behaviors. However, it encounters challenge

In [5]:
!pip install sentence-transformers
!pip install faiss-cpu


In [6]:
######### Create vector Embeddings
from sentence_transformers import SentenceTransformer

# Load an embedding model
embedding_model = SentenceTransformer('all-mpnet-base-v2')

# Generate embeddings for each chunk
chunk_texts = [chunk.page_content for chunk in document_chunks]
embeddings = embedding_model.encode(chunk_texts)

# Example: Print the embedding for the first chunk
print("First Chunk Embedding:", embeddings[0])



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

First Chunk Embedding: [ 4.56623174e-03  2.57609468e-02  3.09704174e-03  2.72877291e-02
 -4.35577556e-02  1.19377524e-02 -1.94524210e-02 -3.08593106e-03
  8.43841862e-03 -5.70919598e-03 -4.45641764e-03  4.74402159e-02
 -4.91559543e-02  2.73134727e-02  1.74758416e-02 -4.98446934e-02
  5.29242530e-02 -5.50772175e-02  2.66424064e-02 -2.17577927e-02
 -2.83069313e-02 -3.29991877e-02 -3.36387195e-02 -1.42266527e-02
  7.45791756e-03 -2.47283783e-02 -9.60575603e-03 -1.23569975e-02
  3.84097248e-02 -1.98398232e-02 -9.36617330e-03  9.49302875e-03
  4.11085971e-02  8.45245086e-03  2.20606512e-06 -7.23988414e-02
 -1.88091602e-02 -4.03939094e-03 -7.61327744e-02 -4.07462567e-02
 -7.20079616e-02  3.21522392e-02 -9.99270100e-03  8.23180750e-03
 -3.91053371e-02  5.86335622e-02  1.32203195e-02  6.09860681e-02
  3.04918289e-02  6.84008449e-02  3.14166322e-02 -4.84430492e-02
  5.47726192e-02 -2.14196146e-02  7.89580569e-02 -4.82406318e-02
 -4.52146865e-03  1.73685681e-02  3.67301740e-02 -2.81777680e-02
 -

In [7]:
import numpy as np
# Convert embeddings to a NumPy array
embeddings_array = np.array(embeddings).astype('float32')
embeddings_array /= np.linalg.norm(embeddings_array, axis=1, keepdims=True)

print(embeddings_array.shape)
np.save("vector_database.npy", embeddings_array)

(419, 768)


In [8]:
######### Saving chunks for later access
import pickle

# Save chunks to a binary file
with open("chunks.pkl", "wb") as f:
    pickle.dump(document_chunks, f)

